In [17]:
pip install opendatasets

Note: you may need to restart the kernel to use updated packages.


In [18]:
import opendatasets as od
import pandas as pd

In [19]:
test_data = pd.read_csv("C:\\Users\\harry\\OneDrive\\Desktop\\Generative AI\\fashionmnist\\fashion-mnist_test.csv")

In [20]:
import torch
import torch.nn as nn
import numpy as np

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [22]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(784, 1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid())
        
    def forward(self, x):
        return self.layer(x)

In [23]:
class Generator(nn.Module):
    def __init__(self, input_dim = 100, output_dim = 784):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, output_dim),
            nn.Tanh())
        
    def forward(self, x):
        return self.layer(x)

In [24]:
train_data = pd.read_csv("C:\\Users\\harry\\OneDrive\\Desktop\\Generative AI\\fashionmnist\\fashion-mnist_train.csv")

In [25]:
train_data.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
df = train_data.drop(columns = ['label'])

In [27]:
print(df.head())

   pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  pixel9  \
0       0       0       0       0       0       0       0       0       0   
1       0       0       0       0       0       0       0       0       0   
2       0       0       0       0       0       0       0       5       0   
3       0       0       0       1       2       0       0       0       0   
4       0       0       0       0       0       0       0       0       0   

   pixel10  ...  pixel775  pixel776  pixel777  pixel778  pixel779  pixel780  \
0        0  ...         0         0         0         0         0         0   
1        0  ...         0         0         0         0         0         0   
2        0  ...         0         0         0        30        43         0   
3        0  ...         3         0         0         0         0         1   
4        0  ...         0         0         0         0         0         0   

   pixel781  pixel782  pixel783  pixel784  
0         0       

In [28]:
from torch.utils.data import DataLoader, Dataset

In [29]:
class MyDataset(Dataset):
    def __init__(self, x):
        self.x = x

    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return torch.tensor(self.x[idx], dtype = torch.float32)

In [30]:
dataset = MyDataset(df.values)
data_loader = DataLoader(dataset, batch_size = 128, shuffle = True, pin_memory = True)

In [ ]:
class Model():
    def __init__(self, batch_size = 128, num_epochs = 10, alpha = 0.001, beta = 0.001):
        self.G_model = Generator()
        self.D_model = Discriminator()
        self.G_opt = torch.optim.Adam(self.G_model.parameters(), lr = alpha)
        self.D_opt = torch.optim.Adam(self.D_model.parameters(), lr = beta)
        self.criterion = nn.BCELoss()
        regularizer = nn.L1Loss()
        self.batch_size = batch_size
        self.num_epochs = num_epochs

    def train(self):
        for i in range(self.num_epochs):
            for i, real_image in enumerate(data_loader):
                real_image = real_image.to(device)
                noise = torch.randn((self.batch_size, 100))
                noise = noise.to(device)

                with torch.no_grad():

                    fake_image = self.G_model(noise)

                D_real_pred = self.D_model(real_image)
                D_fake_pred = self.D_model(fake_image)

                D_real_loss = self.criterion(D_real_pred, torch.ones_like(D_real_pred))
                D_fake_loss = self.criterion(D_fake_pred, torch.zeros_like(D_fake_pred))

                D_loss = 0.5 * (D_real_loss + D_fake_loss)

                self.D_opt.zero_grad()

                D_loss.backward()

                self.D_opt.step()

                fake_image = self.G_model(noise)

                D_fake_pred_for_gan = self.D_model(fake_image)

                G_loss = self.criterion(D_fake_pred_for_gan, torch.ones_like(D_fake_pred_for_gan))

                self.G_opt.zero_grad()

                G_loss.backward()

                self.G_opt.step()

            

In [ ]:
def train(self):
        for i in range(self.num_epochs):
            for i, real_image in enumerate(data_loader):
                real_image = real_image.to(device)
                noise = torch.randn((self.batch_size, 100))
                noise = noise.to(device)

                fake_image = self.G_model(noise)

                D_real_pred = self.D_model(real_image)
                D_fake_pred = self.D_model(fake_image.detach())

                D_real_loss = self.criterion(D_real_pred, torch.ones_like(D_real_pred))
                D_fake_loss = self.criterion(D_fake_pred, torch.zeros_like(D_fake_pred))

                D_loss = 0.5 * (D_real_loss + D_fake_loss)

                self.D_opt.zero_grad()

                D_loss.backward()

                self.D_opt.step()

                D_fake_pred_for_gan = self.D_model(fake_image)

                G_loss = self.criterion(D_fake_pred_for_gan, torch.ones_like(D_fake_pred_for_gan))

                self.G_opt.zero_grad()

                G_loss.backward()

                self.G_opt.step()